<a href="https://colab.research.google.com/github/LoloziAnas/deeplearning/blob/master/PredictingThePriceOfUsedCars_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


In [45]:
# Import the dataset.
dataset = pd.read_csv('toyota.csv')
dataset.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,GT86,2016,16000,Manual,24089,Petrol,265,36.2,2.0
1,GT86,2017,15995,Manual,18615,Petrol,145,36.2,2.0
2,GT86,2015,13998,Manual,27469,Petrol,265,36.2,2.0
3,GT86,2017,18998,Manual,14736,Petrol,150,36.2,2.0
4,GT86,2017,17498,Manual,36284,Petrol,145,36.2,2.0


In [46]:
# Cleaning the data.

print(dataset['transmission'].unique())
# output -> ['Manual' 'Automatic' 'Semi-Auto' 'Other']

#Exclude the case that contains Other.
dataset = dataset[dataset['transmission'] != 'Other']

print(dataset['fuelType'].unique())
# output -> ['Petrol' 'Other' 'Hybrid' 'Diesel']

#Exclude the case that contains Other.
dataset = dataset[dataset['fuelType'] != 'Other']



['Manual' 'Automatic' 'Semi-Auto' 'Other']
['Petrol' 'Other' 'Hybrid' 'Diesel']


In [62]:
# Splitting data to features and target.
X = dataset.iloc[:,[i for i in range(1,9) if  i != 2 ]].values
y = dataset.iloc[:,2].values



(6632, 7)

In [ ]:
from sklearn.preprocessing import LabelEncoder

lEnc_trans = LabelEncoder()
lenc_fType = LabelEncoder()

X[0:1] = lEnc_trans.fit_transform(X[0:1]) #Transmission

X[0:3] = lenc_fType.fit_transform(X[0:3]) #fuelType

X

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

transformcolumn = ColumnTransformer([("transmission",OneHotEncoder(dtype=int, ),['transmission','fuelType'])],remainder='passthrough')

X = transformcolumn.fit_transform(X)
X


In [53]:
# Splitting data into training and test data.
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
y_train = sc.fit_transform(y_train)

In [ ]:

from sklearn import svm
from sklearn.model_selection import GridSearchCV

clf = svm.SVR()
# defining parameter range 
param_grid = {'C': [ i for i in range(1,10)],'gamma': [1, 0.1, 0.01, 0.001, 0.0001],'epsilon':[i*0.1 for i  in range(1,10)],'kernel': ['rbf']}  

parameters = {'C': [1000],'gamma': [0.001],'epsilon':[0.1],'kernel': ['rbf']}  

paramgrid = {"kernel": ["rbf"],
             "C"     : np.logspace(-9, 9, num=25, base=10),
             "gamma" : np.logspace(-9, 9, num=25, base=10)}

regr = GridSearchCV(clf,parameters, refit = True, verbose = 1)

# fitting the model for grid search 
%time regr.fit(x_train,y_train)


In [ ]:
#**Using RandomizedSearchCV**

#Same parameter space, but only test 250 random combinations.
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

regr = RandomizedSearchCV(clf,
                        param_distributions=paramgrid,
                        n_iter=250,
                        cv=StratifiedKFold(n_splits=2),
                        verbose=1)
regr.fit(x_train,y_train)



In [ ]:

# print best parameter after tuning 
print(regr.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(regr.best_estimator_) 


In [ ]:
pred_y_ = regr.predict(x_test)
pred_y_
y_test

regr.score(x_train,y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

print("R2-score: %.2f" % r2_score(pred_y_ , y_test) )
print(" (MSE): %.2f" % mean_squared_error(y_test, pred_y_))
print("Mean absolute error: %.2f" % np.mean(np.absolute(pred_y_ - y_test)))
